In [3]:
!pip install psycopg2-binary

    100% |████████████████████████████████| 1.5MB 7.9MB/s 


In [44]:
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor, Json
import json
import pandas as pd

In [9]:
%run sql_test.py

In [6]:
IP_ADDRESS = '34.222.246.54'
DBNAME = 'postgres'
USER = 'postgres'
PASSWORD = 'foobar1'

In [7]:
def con_cur_to_db(dbname=DBNAME, dict_cur=None):
    ''' 
    Returns both a connection and a cursor object for your database
    '''

    con = pg2.connect(host=IP_ADDRESS, #allows you to navigate db
                  dbname=dbname,
                  user=USER,
                  password=PASSWORD)
    if dict_cur:
        cur = con.cursor(cursor_factory=RealDictCursor)
    else:
        cur = con.cursor()
    return con, cur
    
def execute_query(query, dbname=DBNAME, dict_cur=None, command=False):
    '''
    Executes a query directly to a database, without having to create a cursor and connection each time. 
    '''
    con, cur = con_cur_to_db(dbname, dict_cur)
    cur.execute(f'{query}')
    if not command:
        data = cur.fetchall()
        con.close()
        return data
    con.commit() #sends to server
    con.close() #closes server connection

In [47]:
def insert_entry_json(data, tablename=None):
    con, cur = con_cur_to_db()
    for x in data:
        cur.execute(f'INSERT INTO {tablename} (data) VALUES ({Json(x)});')
    con.commit()
    con.close()

In [8]:
query = '''CREATE TABLE flex_test
(indx SERIAL
data JSONB);'''

In [ ]:
query = '''DROP TABLE flex_test;'''

In [11]:
execute_query(query, command=True)

ProgrammingError: relation "flex_test" already exists


In [ ]:
with open('./tweets_ex.json', 'r') as f:
    raw_data = json.load(f)

In [ ]:
insert_entry_json(raw_data, flex_test)

In [14]:
query = '''SELECT * FROM flex_test;'''

In [15]:
response = execute_query(query, dict_cur=True)

In [17]:
response[0]['data']['text']

'RT @MitzProductions: I’ve been to a shit load of concerts this year but my top 3 in order is:\n\n1. Kung Fu Kenny\n2. Justin Timberlake\n3. Tor…'

In [12]:
execute_query(query)

ProgrammingError: relation "flex_test" already exists


In [54]:
query = """SELECT data->>'text'
FROM flex_test
WHERE data->>'text' IS LIKE 'RT';
"""

In [55]:
response = execute_query(query, dict_cur=True)

ProgrammingError: syntax error at or near "ISLIKE"
LINE 3: WHERE data->>'text' ISLIKE 'RT';
                            ^


In [ ]:
len(response)

In [45]:
df = pd.DataFrame(response)

In [46]:
df.head()

,?column?
0,RT @MitzProductions: I’ve been to a shit load ...
1,Hoje achei nos meus arquivos o áudio de Kind o...
2,47. El show del mediotiempo del Superbowl de e...
3,RT @happilyabby: rt if you stan:\n\n- one dire...
4,na mrl?\njustin timberlake - mirrors
